https://dadataguy.medium.com/using-python-to-transform-data-from-multiple-csv-files-25390da1c187

In [1]:
# importing libraries 
import pandas as pd
import numpy as np

# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x) 
#pd.set_option('display.max_rows', None)

In [3]:
import time
time_begin = time.time()

od =pd.read_csv("order_details.csv")
o  =pd.read_csv("orders.csv")
pt =pd.read_csv("pizza_types.csv",encoding="unicode_escape")
p  =pd.read_csv("pizzas.csv")

print(f'Run time: {round(((time.time()-time_begin)/60),3)} mins')

Run time: 0.001 mins


In [4]:
od.head(2)
o.head(2)
pt.head(2)
p.head(2)

,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,12.75
1,bbq_ckn_m,bbq_ckn,M,16.75


In [6]:
o.isnull().sum()

order_id    0
date        0
time        0
dtype: int64

In [7]:
# Using the built-in string split function. 
pt['ingredients']=pt['ingredients'].str.split(pat = ',')
pt_model = pd.DataFrame(pt['ingredients'].tolist()).fillna('None').add_prefix('items_')
pt = pd.concat([pt, pt_model], axis=1)
pt.head(2)

,pizza_type_id,name,category,ingredients,items_0,items_1,items_2,items_3,items_4,items_5,items_6,items_7
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"[Barbecued Chicken, Red Peppers, Green Peppe...",Barbecued Chicken,Red Peppers,Green Peppers,Tomatoes,Red Onions,Barbecue Sauce,None,None
1,cali_ckn,The California Chicken Pizza,Chicken,"[Chicken, Artichoke, Spinach, Garlic, Jala...",Chicken,Artichoke,Spinach,Garlic,Jalapeno Peppers,Fontina Cheese,Gouda Cheese,None


In [12]:
pd.DataFrame(pt.ingredients.tolist()).add_prefix('item_')

,item_0,item_1,item_2,item_3,item_4,item_5,item_6,item_7
0,Barbecued Chicken,Red Peppers,Green Peppers,Tomatoes,Red Onions,Barbecue Sauce,None,None
1,Chicken,Artichoke,Spinach,Garlic,Jalapeno Peppers,Fontina Cheese,Gouda Cheese,None
2,Chicken,Red Onions,Red Peppers,Mushrooms,Asiago Cheese,Alfredo Sauce,None,None
3,Chicken,Tomatoes,Red Peppers,Spinach,Garlic,Pesto Sauce,None,None
4,Chicken,Tomatoes,Red Peppers,Red Onions,Jalapeno Peppers,Corn,Cilantro,Chipotle Sauce
5,Chicken,Pineapple,Tomatoes,Red Peppers,Thai Sweet Chilli Sauce,None,None,None
6,Bacon,Pepperoni,Italian Sausage,Chorizo Sausage,None,None,None,None
7,Pepperoni,Mushrooms,Red Onions,Red Peppers,Bacon,None,None,None
8,Sliced Ham,Pineapple,Mozzarella Cheese,None,None,None,None,None
9,Capocollo,Red Peppers,Tomatoes,Goat Cheese,Garlic,Oregano,None,None


In [13]:
pt['name']=pt['name'].str[3:]
# double checking my work
pt[pt['pizza_type_id'] == 'hawaiian']

,pizza_type_id,name,category,ingredients,items_0,items_1,items_2,items_3,items_4,items_5,items_6,items_7
8,hawaiian,Hawaiian Pizza,Classic,"[Sliced Ham, Pineapple, Mozzarella Cheese]",Sliced Ham,Pineapple,Mozzarella Cheese,None,None,None,None,None


In [14]:
# Joining the first data frame
df=od.merge(o, how='left', on='order_id')
# Joining the second data frame
df=df.merge(p, how='left', on='pizza_id')
# Joining the third data frame
df=df.merge(pt, how='left', on='pizza_type_id')
# Reviewing the new data frame called df
df.head(2)

,order_details_id,order_id,pizza_id,quantity,date,time,pizza_type_id,size,price,name,category,ingredients,items_0,items_1,items_2,items_3,items_4,items_5,items_6,items_7
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,hawaiian,M,13.25,Hawaiian Pizza,Classic,"[Sliced Ham, Pineapple, Mozzarella Cheese]",Sliced Ham,Pineapple,Mozzarella Cheese,None,None,None,None,None
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,classic_dlx,M,16.00,Classic Deluxe Pizza,Classic,"[Pepperoni, Mushrooms, Red Onions, Red Pepp...",Pepperoni,Mushrooms,Red Onions,Red Peppers,Bacon,None,None,None


In [15]:
# I labeled it CleanedDF, but you can call it whatever you'd like
df.to_csv('CleanedDF.csv', index = False)